# How Delta Lake Supercharges Data Lakes

Delta Lake’s transaction log brings high reliability, performance, and ACID compliant transactions to data lakes. But exactly how does it accomplish this?
Working through concrete examples, we will take a close look at how the transaction logs are managed and leveraged by Delta to supercharge data lakes.

This tutorial notebook was developed using open source Delta Lake in an open source environment.

### Environment used to develop and run this notebook
* [Centos 7.8](https://www.centos.org/download/)
* [Spark 3.0](http://spark.apache.org/docs/latest/)
* [Delta Lake 0.7.0](https://github.com/delta-io/delta/releases)
* [Scala 2.12](https://www.scala-lang.org/download/2.12.8.html)
* [Jupyterlab 2.1.5](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

### Installing Delta Lake
* [Download 0.7.0 jar file](https://mvnrepository.com/artifact/io.delta/delta-core_2.12/0.7.0)
* Move jar file to $SPARK_HOME/jars
* More Details here: [Setting up Apache Spark with Delta](https://docs.delta.io/latest/quick-start.html#set-up-apache-spark-with-delta-lake)

### Source Data for this notebook

The data used in this tutorial is a modified version of the public data from [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Online+Retail#). This dataset contains transactional data from a UK online retailer and it spans January 12, 2010 to September 12, 2011. For a full view of the data please view the data dictionary available [here](http://archive.ics.uci.edu/ml/datasets/Online+Retail#).

<br>
<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>  

An open-source storage format that brings ACID transactions to Apache Spark™ and big data workloads.

* **Open format**: Stored as Parquet format in blob storage.
* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Audit History**: History of all the operations that happened in the table.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box.

<img src="https://www.evernote.com/l/AAF4VIILJtFNZLuvZjGGhZTr2H6Z0wh6rOYB/image.png" width=800px align="center">

# Setup, Helpers, Config & APIs

In [1]:
# Required Classes
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, expr, rand, when, count, col

In [2]:
# Create Spark session & configure it for Delta Lake version 0.7.0

# Load delta library - .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0")
# Enable sql (Delta Lake specific) support within Apache Spark - .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
# Enable integration with Catalog APIs (since 3.0) - .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = SparkSession.builder.appName("DeltaLake Transaction Logs") \
    .master("local[4]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport() \
    .getOrCreate()

# Main class for programmatically intereacting with Delta Tables.
from delta.tables import *

In [3]:
# Set configurations for our Spark Session
# Adjust to your environment, e.g # cores on cluster

spark.conf.set("spark.sql.shuffle.partitions", 4)
spark.conf.set("spark.default.parallelism", 4)

In [4]:
# Load helper functions

%run Helpers.ipynb

# Define Data File Paths

In [5]:
# Source data
# Change paths to match your environment!

inputPath    = "/home/spark/data/source/"
sourceData   = inputPath + "online-retail-dataset.csv"

# Base location for all saved data
basePath     = "/home/spark/data" 

# Path for Parquet formatted data
parquetPath  = basePath + "/parquet/online_retail_data"

# Path for Delta formatted data
deltaPath    = basePath + "/delta/online_retail_data"
deltaLogPath = deltaPath + "/_delta_log"

# Clean up from last run.
! rm -Rf $deltaPath 2>/dev/null
print("Deleted path " + deltaPath)

! rm -Rf $parquetPath 2>/dev/null
print("Deleted path " + parquetPath)

Deleted path /home/spark/data/delta/online_retail_data
Deleted path /home/spark/data/parquet/online_retail_data


# Download and Stage Source Data

In [6]:
# Data sourced from "Spark - The Definitive Guide", located at: https://github.com/databricks/Spark-The-Definitive-Guide
# Data origin: http://archive.ics.uci.edu/ml/datasets/Online+Retail
import os.path

file_exists = os.path.isfile(f'{sourceData}')
 
if not file_exists:
    print("-> Downloading dataset.")
    os.system(f'curl https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/all/online-retail-dataset.csv -o {sourceData}')
    file_exists = os.path.isfile(f'{sourceData}')
    
if file_exists:
    print("-> Dataset is present.\n")
    
    fileSize = ! du -m "$sourceData" | cut -f1 # Posix compliant
    print(f"File [{sourceData}] is {fileSize} MB in size.")

-> Dataset is present.

File [/home/spark/data/source/online-retail-dataset.csv] is ['43'] MB in size.


# Define Schema for Source Data

In [7]:
# Let's take a peek at the data as a text file
! head -n 5 $sourceData 2>/dev/null

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [8]:
# Provide schema for source data
# Schema source: http://archive.ics.uci.edu/ml/datasets/Online+Retail#

# SQL DDL method
schemaDDL = """InvoiceNo Integer, StockCode String, Description String, Quantity Integer, 
               InvoiceDate String, UnitPrice Double, CustomerID Integer, Country String """


# You could also use the StructType method.
# Libraries needed to define schemas
# from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

#inputSchema = StructType([
#  StructField("InvoiceNo", IntegerType(), True),
#  StructField("StockCode", StringType(), True),
#  StructField("Description", StringType(), True),
#  StructField("Quantity", IntegerType(), True),
#  StructField("InvoiceDate", StringType(), True),
#  StructField("UnitPrice", DoubleType(), True),
#  StructField("CustomerID", IntegerType(), True),
#  StructField("Country", StringType(), True)
#])


# Clean and Explore Data

In [9]:
# Create retail sales data dataframe

rawSalesDataDF = (
    spark.read
    .format("csv")
    .option("header","true")
    .schema(schemaDDL)
    .load(sourceData)
)

# Count rows and partitions
rowCount = rawSalesDataDF.count() 
partCount = rawSalesDataDF.rdd.getNumPartitions()

print(f'Row Count: {rowCount} Partition Count: {partCount}')

Row Count: 541909 Partition Count: 4


In [10]:
# Identify columns with null values

print("Columns with null values")
rawSalesDataDF.select([count(when(col(c).isNull(), c)).alias(c) for c in rawSalesDataDF.columns]).show()

Columns with null values
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|     9291|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [11]:
# Remove rows where important columns are null. In our case: InvoiceNo and CustomerID

cleanSalesDataDF = rawSalesDataDF.where(col("InvoiceNo").isNotNull() & col("CustomerID").isNotNull())
cleanSalesDataCount = cleanSalesDataDF.count()
# POO cleanSalesDataDF = cleanSalesDataDF.where(col("CustomerID").isNotNull())

# All rows with null values should be gone
print("null values")
cleanSalesDataDF.select([count(when(col(c).isNull(), c)).alias(c) for c in rawSalesDataDF.columns]).show()

print(f' RowsRemoved: {rowCount-cleanSalesDataCount}\n Final Row Count: {cleanSalesDataCount}')

null values
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+

 RowsRemoved: 143985
 Final Row Count: 397924


In [12]:
# Define new dataframe based on cleansed data but only use a subset of the data to make things run faster

# Random sample of 25%, with seed and without replacement
retailSalesData1 = cleanSalesDataDF.sample(withReplacement=False, fraction=.25, seed=75)

# Count rows and partitions
rowCount = retailSalesData1.count() 
partCount = retailSalesData1.rdd.getNumPartitions()

print(f'Row Count: {rowCount} Partition Count: {partCount}')

Row Count: 99226 Partition Count: 4


In [13]:
# Peek at the dataframe

retailSalesData1.show(3, truncate = False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |21730    |GLASS STAR FROSTED T-LIGHT HOLDER  |6       |12/1/2010 8:26|4.25     |17850     |United Kingdom|
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
only showing top 3 rows



# HIVE Metastore Database Setup

In [14]:
# Create database to hold demo objects
spark.sql("CREATE DATABASE IF NOT EXISTS deltademo")
spark.sql("SHOW DATABASES").show()

# Current DB should be deltademo
spark.sql("USE deltademo")
spark.sql("SELECT CURRENT_DATABASE()").show()
spark.sql("DESCRIBE DATABASE deltademo").show(truncate = False)

# Clean-up from last run
spark.sql("DROP TABLE IF EXISTS SalesParquetFormat")
spark.sql("DROP TABLE IF EXISTS SalesDeltaFormat")
spark.sql("DROP TABLE IF EXISTS tbl_CheckpointFile")
spark.sql("SHOW TABLES").show()


+---------+
|namespace|
+---------+
|  default|
|deltademo|
| louis_db|
+---------+

+------------------+
|current_database()|
+------------------+
|         deltademo|
+------------------+

+-------------------------+----------------------------------------------------------------------------------------+
|database_description_item|database_description_value                                                              |
+-------------------------+----------------------------------------------------------------------------------------+
|Database Name            |deltademo                                                                               |
|Comment                  |                                                                                        |
|Location                 |file:/home/spark/projects/spark3-projects/spark3-playground/spark-warehouse/deltademo.db|
|Owner                    |spark                                                                           

# Working with Parquet Files

In [15]:
# Save data as a table in Parquet format

retailSalesData1.write.saveAsTable('SalesParquetFormat', format='parquet', mode='overwrite',path=parquetPath)

In [16]:
# Let's peek into the catalog and verify that our table was created

spark.catalog.listTables()

# SQL method - not as informative
# spark.sql("show tables").show()

[Table(name='salesparquetformat', database='deltademo', description=None, tableType='EXTERNAL', isTemporary=False)]

In [17]:
# Files and size on disk

files_in_dir(parquetPath, "parquet")

312KB    2020-08-26 10:12:29  part-00001-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
282KB    2020-08-26 10:12:29  part-00000-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
195KB    2020-08-26 10:12:29  part-00003-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
314KB    2020-08-26 10:12:29  part-00002-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet

Number of file/s: 4 | Total size: 1.2M


In [18]:
spark.sql("describe extended SalesParquetFormat").show(100,truncate = False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|InvoiceNo                   |int                                                           |null   |
|StockCode                   |string                                                        |null   |
|Description                 |string                                                        |null   |
|Quantity                    |int                                                           |null   |
|InvoiceDate                 |string                                                        |null   |
|UnitPrice                   |double                                                        |null   |
|CustomerID                  |int                                                 

In [19]:
# Use Spark SQL to query the newly created table

spark.sql("SELECT * FROM SalesParquetFormat;").show(3, truncate = False)

# You can directly query the directory too.
# spark.sql(f"SELECT * FROM parquet.`{parquetPath}` limit 5 ").show(truncate = False)

+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                 |Quantity|InvoiceDate    |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+
|563016   |21497    |FANCY FONTS BIRTHDAY WRAP   |25      |8/11/2011 12:44|0.42     |15358     |United Kingdom|
|563016   |22993    |SET OF 4 PANTRY JELLY MOULDS|12      |8/11/2011 12:44|1.25     |15358     |United Kingdom|
|563016   |22961    |JAM MAKING SET PRINTED      |12      |8/11/2011 12:44|1.45     |15358     |United Kingdom|
+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+
only showing top 3 rows



In [20]:
# Add one row of data to the table
# Parquet being immutable necessitates the creation of an additional Parquet file

spark.sql(""" 
             INSERT INTO SalesParquetFormat
              VALUES(963316, 2291, "WORLD'S BEST JAM MAKING SET", 5, "08/13/2011 07:58", 1.45, 15358, "United Kingdom")
          """)

DataFrame[]

In [21]:
files_in_dir(parquetPath, "parquet")

312KB    2020-08-26 10:12:29  part-00001-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
282KB    2020-08-26 10:12:29  part-00000-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
195KB    2020-08-26 10:12:29  part-00003-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
314KB    2020-08-26 10:12:29  part-00002-7c5ca58b-c85f-4fc7-826b-35886eebec8d-c000.snappy.parquet
2KB      2020-08-26 10:12:46  part-00000-598a98d4-a17f-4545-852e-85489289f836-c000.snappy.parquet

Number of file/s: 5 | Total size: 1.2M


In [ ]:
# Let's have a peek at the new file

# spark.read.load(f"{parquetPath}/part-00000-806d3f4d-6f4e-4d04-a464-f08a1cda3b2f-c000.snappy.parquet").show(truncate = False)

- Above we saw how to create a table structure using Parquet as the underlying data file format.<br>
- Using sql we were able to query the table and even insert new data using sql.<br>
- However, no history was kept of these operations.<br>

# Working with Delta Lake

<img src="https://files.training.databricks.com/images/adbcore/AAFxQkg_SzRC06GvVeatDBnNbDL7wUUgCg4B.png" alt="Delta Lake" width="600" align="left"/>

In [22]:
# Save retailSalesData1 to Delta

retailSalesData1.write.mode("overwrite").format("delta").save(deltaPath)

# Query delta directory directly
spark.sql(f"SELECT * FROM delta.`{deltaPath}` limit 3 ").show(truncate = False)

+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                 |Quantity|InvoiceDate    |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+
|563016   |21497    |FANCY FONTS BIRTHDAY WRAP   |25      |8/11/2011 12:44|0.42     |15358     |United Kingdom|
|563016   |22993    |SET OF 4 PANTRY JELLY MOULDS|12      |8/11/2011 12:44|1.25     |15358     |United Kingdom|
|563016   |22961    |JAM MAKING SET PRINTED      |12      |8/11/2011 12:44|1.45     |15358     |United Kingdom|
+---------+---------+----------------------------+--------+---------------+---------+----------+--------------+



In [23]:
# Create variable for a path based Delta table

deltaTable = DeltaTable.forPath(spark, deltaPath)

In [24]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
history = deltaTable.history().select('version','timestamp','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(truncate = False)

####### HISTORY ########
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+
|ver|timestamp          |operation|operationParameters                   |operationMetrics                                                  |
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+
|0  |2020-08-26 10:13:04|WRITE    |[mode -> Overwrite, partitionBy -> []]|[numFiles -> 4, numOutputBytes -> 1129731, numOutputRows -> 99226]|
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+



In [27]:
# files and size on disk
# Notice the sub-directory "_delta_log"

! ls -thl $deltaPath

total 1.1M
drwxrwxr-x. 2 spark spark   39 Aug 26 10:13 _delta_log
-rw-r--r--. 1 spark spark 282K Aug 26 10:13 part-00000-30f975b9-52a0-4eb4-b72e-1ce6669d9576-c000.snappy.parquet
-rw-r--r--. 1 spark spark 313K Aug 26 10:13 part-00001-58b422df-e828-45a4-b95c-ac2f62ee2dc7-c000.snappy.parquet
-rw-r--r--. 1 spark spark 315K Aug 26 10:13 part-00002-25235bcc-3dc3-4a62-b354-10ca68c42a83-c000.snappy.parquet
-rw-r--r--. 1 spark spark 196K Aug 26 10:13 part-00003-e1c13592-efaf-4ca7-9a0f-25484dccc18d-c000.snappy.parquet


In [28]:
# # files and size on disk
# We can see that parquet files were added but now there is a trx log

files_in_dir(deltaLogPath, "json")

2KB      2020-08-26 10:13:04  00000000000000000000.json

Number of file/s: 1 | Total size: 4.0K


In [29]:
# Let's have a peek inside the trx log

spark.read.format("json").load(deltaLogPath + "/00000000000000000000.json").collect()

[Row(add=None, commitInfo=Row(isBlindAppend=False, operation='WRITE', operationMetrics=Row(numFiles='4', numOutputBytes='1129731', numOutputRows='99226'), operationParameters=Row(mode='Overwrite', partitionBy='[]'), timestamp=1598451184123), metaData=None, protocol=None),
 Row(add=None, commitInfo=None, metaData=None, protocol=Row(minReaderVersion=1, minWriterVersion=2)),
 Row(add=None, commitInfo=None, metaData=Row(createdTime=1598451182505, format=Row(provider='parquet'), id='4126e7fa-86b6-4189-b565-79b85f699224', partitionColumns=[], schemaString='{"type":"struct","fields":[{"name":"InvoiceNo","type":"integer","nullable":true,"metadata":{}},{"name":"StockCode","type":"string","nullable":true,"metadata":{}},{"name":"Description","type":"string","nullable":true,"metadata":{}},{"name":"Quantity","type":"integer","nullable":true,"metadata":{}},{"name":"InvoiceDate","type":"string","nullable":true,"metadata":{}},{"name":"UnitPrice","type":"double","nullable":true,"metadata":{}},{"name":"

In [30]:
# Create a new dataframe with fraction of original data.
# Random sample of 25%, with seed and without replacement

retailSalesData2 = cleanSalesDataDF.sample(withReplacement=False, fraction=.25, seed=31)
retailSalesData2.count()

99149

In [31]:
# Add to our Delta Lake table by appending retailSalesData2

retailSalesData2.write.mode("append").format("delta").save(deltaPath)

In [32]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
# Reference for full history schema: https://docs.delta.io/latest/delta-utility.html
history = deltaTable.history().select('version','timestamp','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(truncate = False)

####### HISTORY ########
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+
|ver|timestamp          |operation|operationParameters                   |operationMetrics                                                  |
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+
|1  |2020-08-26 10:14:14|WRITE    |[mode -> Append, partitionBy -> []]   |[numFiles -> 4, numOutputBytes -> 1125004, numOutputRows -> 99149]|
|0  |2020-08-26 10:13:04|WRITE    |[mode -> Overwrite, partitionBy -> []]|[numFiles -> 4, numOutputBytes -> 1129731, numOutputRows -> 99226]|
+---+-------------------+---------+--------------------------------------+------------------------------------------------------------------+



In [33]:
# Data Files and size on disk

files_in_dir(deltaPath, "parquet")

195KB    2020-08-26 10:13:03  part-00003-e1c13592-efaf-4ca7-9a0f-25484dccc18d-c000.snappy.parquet
314KB    2020-08-26 10:13:03  part-00002-25235bcc-3dc3-4a62-b354-10ca68c42a83-c000.snappy.parquet
312KB    2020-08-26 10:13:03  part-00001-58b422df-e828-45a4-b95c-ac2f62ee2dc7-c000.snappy.parquet
282KB    2020-08-26 10:13:03  part-00000-30f975b9-52a0-4eb4-b72e-1ce6669d9576-c000.snappy.parquet
193KB    2020-08-26 10:14:13  part-00003-5e6f6655-260d-4c02-9539-517d20cc04c4-c000.snappy.parquet
281KB    2020-08-26 10:14:13  part-00000-0399af6a-0418-4beb-a296-b74b1563bda6-c000.snappy.parquet
315KB    2020-08-26 10:14:14  part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet
310KB    2020-08-26 10:14:14  part-00001-5f7935a6-3fb4-469f-97c0-cfd66dec38a6-c000.snappy.parquet

Number of file/s: 8 | Total size: 2.3M


In [34]:
# Transaction logs and size on disk

files_in_dir(deltaLogPath, "json")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json

Number of file/s: 2 | Total size: 8.0K


In [35]:
# Peek inside the new transaction log

logDF = spark.read.format("json").load(deltaLogPath + "/00000000000000000001.json")
logDF.collect()

[Row(add=None, commitInfo=Row(isBlindAppend=True, operation='WRITE', operationMetrics=Row(numFiles='4', numOutputBytes='1125004', numOutputRows='99149'), operationParameters=Row(mode='Append', partitionBy='[]'), readVersion=0, timestamp=1598451254014)),
 Row(add=Row(dataChange=True, modificationTime=1598451253000, path='part-00000-0399af6a-0418-4beb-a296-b74b1563bda6-c000.snappy.parquet', size=287535), commitInfo=None),
 Row(add=Row(dataChange=True, modificationTime=1598451254000, path='part-00001-5f7935a6-3fb4-469f-97c0-cfd66dec38a6-c000.snappy.parquet', size=317603), commitInfo=None),
 Row(add=Row(dataChange=True, modificationTime=1598451254000, path='part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet', size=322207), commitInfo=None),
 Row(add=Row(dataChange=True, modificationTime=1598451253000, path='part-00003-5e6f6655-260d-4c02-9539-517d20cc04c4-c000.snappy.parquet', size=197659), commitInfo=None)]

In [36]:
# Create SQL table to make life easier
# Stick with SQL from here on out, where possible.

spark.sql("""
    DROP TABLE IF EXISTS SalesDeltaFormat
  """)
spark.sql("""
    CREATE TABLE SalesDeltaFormat
    USING DELTA
    LOCATION '{}'
  """.format(deltaPath))

DataFrame[]

In [37]:
# Let's peek into the catalog and verify that our table was created.
spark.catalog.listTables()

[Table(name='salesdeltaformat', database='deltademo', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='salesparquetformat', database='deltademo', description=None, tableType='EXTERNAL', isTemporary=False)]

In [38]:
spark.sql("describe extended SalesDeltaFormat").show(100, truncate = False)

+----------------------------+----------------------------------------------+-------+
|col_name                    |data_type                                     |comment|
+----------------------------+----------------------------------------------+-------+
|InvoiceNo                   |int                                           |       |
|StockCode                   |string                                        |       |
|Description                 |string                                        |       |
|Quantity                    |int                                           |       |
|InvoiceDate                 |string                                        |       |
|UnitPrice                   |double                                        |       |
|CustomerID                  |int                                           |       |
|Country                     |string                                        |       |
|                            |                        

In [39]:
# Let's find a Invoice with only 1 count and use it to test DML.
oneRandomInvoice = spark.sql(""" SELECT InvoiceNo, count(*)
                                 FROM SalesDeltaFormat
                                 GROUP BY InvoiceNo
                                 ORDER BY 2 asc
                                 LIMIT 1
                             """).collect()[0][0]

print(f"Random Invoice # => {oneRandomInvoice}")

Random Invoice # => 563202


In [40]:
# Before DML (insert)

spark.sql(f"""
              SELECT SUBSTRING(input_file_name(), -67, 67) AS FileName,
                     * FROM SalesDeltaFormat 
              WHERE InvoiceNo = {oneRandomInvoice}
           """).show(truncate = False)

+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------------+---------+----------+-------+
|FileName                                                           |InvoiceNo|StockCode|Description                |Quantity|InvoiceDate    |UnitPrice|CustomerID|Country|
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------------+---------+----------+-------+
|part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet|563202   |85066    |CREAM SWEETHEART MINI CHEST|1       |8/14/2011 11:16|12.75    |12722     |France |
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------------+---------+----------+-------+



In [41]:
# Let's add some data to our table

spark.sql(f"""
               INSERT INTO SalesDeltaFormat
               VALUES({oneRandomInvoice}, 2291, "WORLD'S BEST JAM MAKING SET", 5, "08/13/2011 07:58", 1.45, 15358, "France");
          """)

DataFrame[]

In [42]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json

Number of file/s: 3 | Total size: 12K


In [43]:
# Schema details: https://docs.delta.io/latest/delta-utility.html

logDF = spark.read.format("json").load(deltaLogPath + "/00000000000000000002.json")
#dfLog.printSchema()
logDF.collect()

[Row(add=None, commitInfo=Row(isBlindAppend=True, operation='WRITE', operationMetrics=Row(numFiles='1', numOutputBytes='2369', numOutputRows='1'), operationParameters=Row(mode='Append', partitionBy='[]'), readVersion=1, timestamp=1598451299827)),
 Row(add=Row(dataChange=True, modificationTime=1598451299000, path='part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet', size=2369), commitInfo=None)]

In [44]:
# After DML (insert)

spark.sql(f"""
              SELECT SUBSTRING(input_file_name(), -67, 67) AS FileName, *
                     FROM SalesDeltaFormat 
                     WHERE InvoiceNo = {oneRandomInvoice}
           """).show(truncate = False)

+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|FileName                                                           |InvoiceNo|StockCode|Description                |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country|
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet|563202   |85066    |CREAM SWEETHEART MINI CHEST|1       |8/14/2011 11:16 |12.75    |12722     |France |
|part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet|563202   |2291     |WORLD'S BEST JAM MAKING SET|5       |08/13/2011 07:58|1.45     |15358     |France |
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------

In [45]:
# Update one invoice

spark.sql(f"""
              UPDATE SalesDeltaFormat
              SET Quantity = Quantity + 1000
              WHERE InvoiceNo = {oneRandomInvoice}
           """)

#deltaTable.update(
#    condition=("InvoiceNo = oneRandomInvoice"),
#    set={"Quantity": expr("Quantity + 1000")}
#)

DataFrame[]

In [46]:
# After Update

spark.sql(f"""
              SELECT 
              SUBSTRING(input_file_name(), -67, 67) AS FileName, *
              FROM SalesDeltaFormat 
              WHERE InvoiceNo = {oneRandomInvoice}
           """).show(truncate = False)

+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|FileName                                                           |InvoiceNo|StockCode|Description                |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country|
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|part-00000-ba14adf0-b9e9-4ce8-9eff-d39bf1ad4d55-c000.snappy.parquet|563202   |85066    |CREAM SWEETHEART MINI CHEST|1001    |8/14/2011 11:16 |12.75    |12722     |France |
|part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet|563202   |2291     |WORLD'S BEST JAM MAKING SET|1005    |08/13/2011 07:58|1.45     |15358     |France |
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------

In [47]:
print("####### HISTORY ########")

# Show which datafile was removed.

# Observe history of actions taken on a Delta table
# spark.sql not supported in 0.7.0 OSS Delta
history = deltaTable.history().select('version','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(truncate = False)

####### HISTORY ########
+---+---------+----------------------------------------+---------------------------------------------------------------------------------------+
|ver|operation|operationParameters                     |operationMetrics                                                                       |
+---+---------+----------------------------------------+---------------------------------------------------------------------------------------+
|3  |UPDATE   |[predicate -> (InvoiceNo#2642 = 563202)]|[numRemovedFiles -> 2, numAddedFiles -> 2, numUpdatedRows -> 2, numCopiedRows -> 29509]|
|2  |WRITE    |[mode -> Append, partitionBy -> []]     |[numFiles -> 1, numOutputBytes -> 2369, numOutputRows -> 1]                            |
|1  |WRITE    |[mode -> Append, partitionBy -> []]     |[numFiles -> 4, numOutputBytes -> 1125004, numOutputRows -> 99149]                     |
|0  |WRITE    |[mode -> Overwrite, partitionBy -> []]  |[numFiles -> 4, numOutputBytes -> 1129731, numOut

In [48]:
files_in_dir(deltaPath, "parquet")

195KB    2020-08-26 10:13:03  part-00003-e1c13592-efaf-4ca7-9a0f-25484dccc18d-c000.snappy.parquet
314KB    2020-08-26 10:13:03  part-00002-25235bcc-3dc3-4a62-b354-10ca68c42a83-c000.snappy.parquet
312KB    2020-08-26 10:13:03  part-00001-58b422df-e828-45a4-b95c-ac2f62ee2dc7-c000.snappy.parquet
282KB    2020-08-26 10:13:03  part-00000-30f975b9-52a0-4eb4-b72e-1ce6669d9576-c000.snappy.parquet
193KB    2020-08-26 10:14:13  part-00003-5e6f6655-260d-4c02-9539-517d20cc04c4-c000.snappy.parquet
281KB    2020-08-26 10:14:13  part-00000-0399af6a-0418-4beb-a296-b74b1563bda6-c000.snappy.parquet
315KB    2020-08-26 10:14:14  part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet
310KB    2020-08-26 10:14:14  part-00001-5f7935a6-3fb4-469f-97c0-cfd66dec38a6-c000.snappy.parquet
2KB      2020-08-26 10:14:59  part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet
2KB      2020-08-26 10:15:18  part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet
315KB    2020-08-26 

In [49]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json
902B     2020-08-26 10:15:18  00000000000000000003.json

Number of file/s: 4 | Total size: 16K


In [50]:
# Schema details: https://docs.delta.io/latest/delta-utility.html

logDF = spark.read.format("json").load(deltaLogPath + "/00000000000000000003.json")
#dfLog.printSchema()
logDF.collect()

[Row(add=None, commitInfo=Row(isBlindAppend=False, operation='UPDATE', operationMetrics=Row(numAddedFiles='2', numCopiedRows='29509', numRemovedFiles='2', numUpdatedRows='2'), operationParameters=Row(predicate='(InvoiceNo#2642 = 563202)'), readVersion=2, timestamp=1598451318801), remove=None),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451318624, path='part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet')),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451318624, path='part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet')),
 Row(add=Row(dataChange=True, modificationTime=1598451318000, path='part-00000-ba14adf0-b9e9-4ce8-9eff-d39bf1ad4d55-c000.snappy.parquet', size=322239), commitInfo=None, remove=None),
 Row(add=Row(dataChange=True, modificationTime=1598451318000, path='part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet', size=2369), commitInfo=None, remove=None)]

In [51]:
# Before DML (delete)

spark.sql(f"""select 
          substring(input_file_name(), -67, 67) as FileName,
          * from SalesDeltaFormat 
          where InvoiceNo = {oneRandomInvoice}""").show(truncate = False)

+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|FileName                                                           |InvoiceNo|StockCode|Description                |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country|
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+----------------+---------+----------+-------+
|part-00000-ba14adf0-b9e9-4ce8-9eff-d39bf1ad4d55-c000.snappy.parquet|563202   |85066    |CREAM SWEETHEART MINI CHEST|1001    |8/14/2011 11:16 |12.75    |12722     |France |
|part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet|563202   |2291     |WORLD'S BEST JAM MAKING SET|1005    |08/13/2011 07:58|1.45     |15358     |France |
+-------------------------------------------------------------------+---------+---------+---------------------------+--------+---------

In [52]:
# https://github.com/delta-io/delta/blob/master/examples/python/quickstart.py
# Delete and invoice (two records)

# This results in one new file being created.  One file had just the one record so it does not have to be re-created
# Each of the two records were in two different files. One of those files had only one record so it did not have to be re-created.

spark.sql(f"DELETE FROM SalesDeltaFormat WHERE InvoiceNo = {oneRandomInvoice}")

# deltaTable.delete(
#    condition=("InvoiceNo = {537617}")
# )

DataFrame[]

In [53]:
# After DML (delete)

spark.sql(f"""
              SELECT 
              SUBSTRING(input_file_name(), -67, 67) as FileName, *
              FROM SalesDeltaFormat 
              WHERE InvoiceNo = {oneRandomInvoice}
          """).show(truncate = False)

+--------+---------+---------+-----------+--------+-----------+---------+----------+-------+
|FileName|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+--------+---------+---------+-----------+--------+-----------+---------+----------+-------+
+--------+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [54]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
history = deltaTable.history().select('version','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(truncate = False)

####### HISTORY ########
+---+---------+----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+
|ver|operation|operationParameters                                                               |operationMetrics                                                                       |
+---+---------+----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+
|4  |DELETE   |[predicate -> ["(spark_catalog.deltademo.SalesDeltaFormat.`InvoiceNo` = 563202)"]]|[numRemovedFiles -> 2, numDeletedRows -> 2, numAddedFiles -> 1, numCopiedRows -> 29509]|
|3  |UPDATE   |[predicate -> (InvoiceNo#2642 = 563202)]                                          |[numRemovedFiles -> 2, numAddedFiles -> 2, numUpdatedRows -> 2, numCopiedRows -> 29509]|
|2  |WRITE    |[mode -> Append, partitio

In [55]:
files_in_dir(deltaPath, "parquet")

195KB    2020-08-26 10:13:03  part-00003-e1c13592-efaf-4ca7-9a0f-25484dccc18d-c000.snappy.parquet
314KB    2020-08-26 10:13:03  part-00002-25235bcc-3dc3-4a62-b354-10ca68c42a83-c000.snappy.parquet
312KB    2020-08-26 10:13:03  part-00001-58b422df-e828-45a4-b95c-ac2f62ee2dc7-c000.snappy.parquet
282KB    2020-08-26 10:13:03  part-00000-30f975b9-52a0-4eb4-b72e-1ce6669d9576-c000.snappy.parquet
193KB    2020-08-26 10:14:13  part-00003-5e6f6655-260d-4c02-9539-517d20cc04c4-c000.snappy.parquet
281KB    2020-08-26 10:14:13  part-00000-0399af6a-0418-4beb-a296-b74b1563bda6-c000.snappy.parquet
315KB    2020-08-26 10:14:14  part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet
310KB    2020-08-26 10:14:14  part-00001-5f7935a6-3fb4-469f-97c0-cfd66dec38a6-c000.snappy.parquet
2KB      2020-08-26 10:14:59  part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet
2KB      2020-08-26 10:15:18  part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet
315KB    2020-08-26 

In [56]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json
902B     2020-08-26 10:15:18  00000000000000000003.json
775B     2020-08-26 10:16:31  00000000000000000004.json

Number of file/s: 5 | Total size: 20K


In [57]:
logDF = spark.read.format("json").load(deltaLogPath + "/00000000000000000004.json")
#dfLog.printSchema()
logDF.collect()

[Row(add=None, commitInfo=Row(isBlindAppend=False, operation='DELETE', operationMetrics=Row(numAddedFiles='1', numCopiedRows='29509', numDeletedRows='2', numRemovedFiles='2'), operationParameters=Row(predicate='["(spark_catalog.deltademo.SalesDeltaFormat.`InvoiceNo` = 563202)"]'), readVersion=3, timestamp=1598451391965), remove=None),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451391963, path='part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet')),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451391963, path='part-00000-ba14adf0-b9e9-4ce8-9eff-d39bf1ad4d55-c000.snappy.parquet')),
 Row(add=Row(dataChange=True, modificationTime=1598451391000, path='part-00000-54540c53-d90d-4661-bc10-f81f739e556f-c000.snappy.parquet', size=322988), commitInfo=None, remove=None)]

In [58]:
# Randomy update 5 random invoices to force a checkpoint

count = 0
anInvoice = retailSalesData2.select("InvoiceNo").orderBy(rand()).limit(1).collect()[0][0]

while (count <= 5):
  deltaTable.update(
    condition=(f"InvoiceNo = {anInvoice}"),
    set={"Quantity": expr("Quantity + 100")})

  count = count + 1
  anInvoice = retailSalesData2.select("InvoiceNo").orderBy(rand()).limit(1).collect()[0][0]

In [59]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json
902B     2020-08-26 10:15:18  00000000000000000003.json
775B     2020-08-26 10:16:31  00000000000000000004.json
905B     2020-08-26 10:17:59  00000000000000000005.json
904B     2020-08-26 10:18:00  00000000000000000006.json
905B     2020-08-26 10:18:02  00000000000000000007.json
905B     2020-08-26 10:18:04  00000000000000000008.json
905B     2020-08-26 10:18:05  00000000000000000009.json
905B     2020-08-26 10:18:07  00000000000000000010.json
17KB     2020-08-26 10:18:08  00000000000000000010.checkpoint.parquet

Number of file/s: 12 | Total size: 76K


In [60]:
checkPointDF = spark.read.format("parquet").load(deltaLogPath + "/00000000000000000010.checkpoint.parquet")
checkPointDF.show(100, truncate = False)

+----+---------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+----------+
|txn |add                                                                                                      |remove       

In [61]:
checkPointFile10 =(
    checkPointDF.select(col("add.path").alias("FileAdded"),
                        col("add.modificationTime").alias("DateAdded"),
                        col("remove.path").alias("FileDeleted"),
                        col("remove.deletionTimestamp").alias("DateDeleted"))
                .orderBy(["DateAdded","DateDeleted"], ascending=[True,False])
)

spark.sql("DROP TABLE IF EXISTS tbl_checkpointfile")
spark.sql("CREATE TABLE IF NOT EXISTS tbl_checkpointfile (Action string, filename string, ActionDate Long)")

checkPointFile10.createOrReplaceTempView("vw_checkpointfile")

In [62]:
spark.sql("select * from vw_checkpointfile limit 100").show(100, truncate=False)

+-------------------------------------------------------------------+-------------+-------------------------------------------------------------------+-------------+
|FileAdded                                                          |DateAdded    |FileDeleted                                                        |DateDeleted  |
+-------------------------------------------------------------------+-------------+-------------------------------------------------------------------+-------------+
|null                                                               |null         |part-00001-6bbaab90-8346-44c2-9b63-154772010091-c000.snappy.parquet|1598451487230|
|null                                                               |null         |part-00000-a79e958c-d460-484b-ba1a-8cba5893c15d-c000.snappy.parquet|1598451487230|
|null                                                               |null         |part-00001-150dba95-8b21-4e5e-bce5-6725dcce2f28-c000.snappy.parquet|1598451485496|
|nul

In [63]:
spark.sql("""
          INSERT INTO tbl_checkpointfile
          SELECT "Add", FileAdded, DateAdded
          FROM vw_checkpointfile
          WHERE FileAdded IS NOT NULL
          """)

spark.sql("""
          INSERT INTO tbl_checkpointfile
          SELECT "Delete", FileDeleted, DateDeleted
          FROM vw_checkpointfile
          WHERE FileDeleted IS NOT NULL
          """)

DataFrame[]

In [64]:
spark.sql("""
           SELECT Action, 
                  filename as `File Name`, 
                  from_unixtime(actiondate/1e3) AS `ActionDate`
           FROM tbl_checkpointfile 
           order by ActionDate asc
          """).show(200, truncate=False)

+------+-------------------------------------------------------------------+-------------------+
|Action|File Name                                                          |ActionDate         |
+------+-------------------------------------------------------------------+-------------------+
|Add   |part-00003-e1c13592-efaf-4ca7-9a0f-25484dccc18d-c000.snappy.parquet|2020-08-26 10:13:03|
|Add   |part-00003-5e6f6655-260d-4c02-9539-517d20cc04c4-c000.snappy.parquet|2020-08-26 10:14:13|
|Delete|part-00000-e061bc4e-947b-4a4b-9cef-1fc47c0598ea-c000.snappy.parquet|2020-08-26 10:15:18|
|Delete|part-00002-454272fb-01fb-4adc-860a-8cb040e4ec52-c000.snappy.parquet|2020-08-26 10:15:18|
|Delete|part-00000-ba14adf0-b9e9-4ce8-9eff-d39bf1ad4d55-c000.snappy.parquet|2020-08-26 10:16:31|
|Delete|part-00001-0eba87ae-1189-4cbd-b3ea-3ece7812113e-c000.snappy.parquet|2020-08-26 10:16:31|
|Delete|part-00002-25235bcc-3dc3-4a62-b354-10ca68c42a83-c000.snappy.parquet|2020-08-26 10:17:58|
|Delete|part-00000-54540c53-d9

In [65]:
# Let's add some data to our table by doing a merge
# Do this to show how json logs pick up after checkpoint

# Create a tiny dataframe to use with merge
mergeSalesData= cleanSalesDataDF.sample(withReplacement=False, fraction=.0001, seed=13)
mergeSalesData.createOrReplaceTempView("vw_mergeSalesData")

# User-defined commit metadata
spark.sql(f"""
               SET spark.databricks.delta.commitInfo.userMetadata=08-25-2020 Data Merge;
          """)

spark.sql("""
          MERGE INTO SalesDeltaFormat
          USING vw_mergeSalesData
          ON SalesDeltaFormat.StockCode = vw_mergeSalesData.StockCode
           AND SalesDeltaFormat.InvoiceNo = vw_mergeSalesData.InvoiceNo
          WHEN MATCHED THEN
            UPDATE SET *
          WHEN NOT MATCHED
            THEN INSERT *
          """)


DataFrame[]

In [66]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
history = deltaTable.history().select('version','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(truncate = False)

####### HISTORY ########
+---+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ver|operation|operationParameters                                                                                                                                                                               |operationMetrics                                                                                                                                                                                                       |
+---+---------+----------------------------------------------------------------------------------------------------------

In [67]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json
902B     2020-08-26 10:15:18  00000000000000000003.json
775B     2020-08-26 10:16:31  00000000000000000004.json
905B     2020-08-26 10:17:59  00000000000000000005.json
904B     2020-08-26 10:18:00  00000000000000000006.json
905B     2020-08-26 10:18:02  00000000000000000007.json
905B     2020-08-26 10:18:04  00000000000000000008.json
905B     2020-08-26 10:18:05  00000000000000000009.json
905B     2020-08-26 10:18:07  00000000000000000010.json
17KB     2020-08-26 10:18:08  00000000000000000010.checkpoint.parquet
2KB      2020-08-26 10:19:36  00000000000000000011.json

Number of file/s: 13 | Total size: 80K


In [68]:
logDF = spark.read.format("json").load(deltaLogPath + "/00000000000000000011.json")
#dfLog.printSchema()
logDF.collect()

[Row(add=None, commitInfo=Row(isBlindAppend=False, operation='MERGE', operationMetrics=Row(numOutputRows='198400', numSourceRows='46', numTargetFilesAdded='4', numTargetFilesRemoved='8', numTargetRowsCopied='198349', numTargetRowsDeleted='0', numTargetRowsInserted='26', numTargetRowsUpdated='25'), operationParameters=Row(predicate='((spark_catalog.deltademo.SalesDeltaFormat.`StockCode` = vw_mergesalesdata.`StockCode`) AND (spark_catalog.deltademo.SalesDeltaFormat.`InvoiceNo` = vw_mergesalesdata.`InvoiceNo`))'), readVersion=10, timestamp=1598451576336, userMetadata='08-25-2020 Data Merge;'), remove=None),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451576332, path='part-00001-93d4d843-87fb-456c-89c5-3c7db9f9e99d-c000.snappy.parquet')),
 Row(add=None, commitInfo=None, remove=Row(dataChange=True, deletionTimestamp=1598451576335, path='part-00000-4c305d94-5f07-4ed5-b0fc-43216acbdba7-c000.snappy.parquet')),
 Row(add=None, commitInfo=None, remove=Row(dat

In [69]:
# Count files in deltaPath

numFiles = ! ls $deltaPath/*parquet | wc -l
print(f"There are {numFiles} files.")

There are ['28'] files.


In [73]:
spark.sql("""
           SELECT Action, 
                  COUNT(Action) as `CountAction`
           FROM tbl_checkpointfile 
           GROUP BY ACTION WITH ROLLUP
          """).show(200, truncate=False)

+------+-----------+
|ACTION|CountAction|
+------+-----------+
|Add   |8          |
|Delete|16         |
|null  |24         |
+------+-----------+



# Delta Lake File Compaction


In [74]:
# Create an artificial "small file" problem

(spark.read
.format("delta")
.load(deltaPath)
.repartition(1000)
.write
.option("dataChange", True)
.format("delta")
.mode("overwrite")
.save(deltaPath)
)


In [75]:
# Count files in deltaPath

numFiles = ! ls $deltaPath/*parquet | wc -l
print(f"There are {numFiles} files.")

There are ['1028'] files.


In [76]:
%%time

# spark.sql("select * from SalesDeltaFormat limit 2").show()

rowCount = spark.sql(""" SELECT CustomerID, count(Country) AS num_countries
                         FROM SalesDeltaFormat
                         GROUP BY CustomerID 
                     """).count()

print(f"Row Count => {rowCount}\n")

Row Count => 4244

CPU times: user 0 ns, sys: 3.56 ms, total: 3.56 ms
Wall time: 987 ms


In [77]:
# Compact 1000 files to 4

(spark.read
.format("delta")
.load(deltaPath)
.repartition(4)
.write
.option("dataChange", False)
.format("delta")
.mode("overwrite")
.save(deltaPath)
)

In [78]:
# Count files in deltaPath

numFiles = ! ls $deltaPath/*parquet | wc -l
print(f"There are {numFiles} files.")

There are ['1032'] files.


In [79]:
%%time

# spark.sql("select * from SalesDeltaFormat limit 2").show()
rowCount = spark.sql(""" select CustomerID, count(Country) as num_countries
                         from SalesDeltaFormat
                        group by CustomerID """).count()

print(f"Row Count => {rowCount}")

Row Count => 4244
CPU times: user 596 µs, sys: 3.16 ms, total: 3.76 ms
Wall time: 255 ms


# Delta Time Travel Queries

In [80]:
# Time Travel Queries

#POO currentVersion = deltaTable.history(1).select("version").collect()[0][0]
# Determine latest version of the Delta table
currentVersion = spark.sql("DESCRIBE HISTORY SalesDeltaFormat LIMIT 1").collect()[0][0]

# Query table as of the current version to attain row count
currentRowCount = spark.read.format("delta").option("versionAsOf", currentVersion).load(deltaPath).count()

print(f"Row Count: {currentRowCount} as of table version {currentVersion}")
print("")

Row Count: 198400 as of table version 13



In [81]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
history = deltaTable.history().select('version','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(100, truncate = False)

####### HISTORY ########
+---+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ver|operation|operationParameters                                                                                                                                                                               |operationMetrics                                                                                                                                                                                                       |
+---+---------+----------------------------------------------------------------------------------------------------------

In [82]:
# Determine difference in record count between the current version and the original version of the table.

origRowCount = spark.read.format("delta").option("versionAsOf", 0).load(deltaPath).count()
print(f"There are {currentRowCount-origRowCount} more rows in version [{currentVersion}] than version [0] of the table.")

There are 99174 more rows in version [13] than version [0] of the table.


In [83]:
# Roll back current table to version 0 (original).
(
    spark
    .read
    .format("delta")
    .option("versionAsOf",0)
    .load(deltaPath)
    .write
    .format("delta")
    .mode("overwrite")
    .save(deltaPath)
)

In [84]:
# Current version should have same record count as version 0.

currentVersion = spark.sql("DESCRIBE HISTORY SalesDeltaFormat LIMIT 1").collect()[0][0]
# If equal it will return "true"
spark.read.format("delta").option("versionAsOf", currentVersion).load(deltaPath).count() == spark.read.format("delta").option("versionAsOf", 0).load(deltaPath).count()

True

In [85]:
print("####### HISTORY ########")

# Observe history of actions taken on a Delta table
history = deltaTable.history().select('version','operation', 'operationParameters', \
                                      'operationMetrics') \
                              .withColumnRenamed("version", "ver")

history.show(100, truncate = False)

####### HISTORY ########
+---+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ver|operation|operationParameters                                                                                                                                                                               |operationMetrics                                                                                                                                                                                                       |
+---+---------+----------------------------------------------------------------------------------------------------------

# Delta Vacuum - Data Retention

delta.logRetentionDuration - default 30 days
<br>
delta.deletedFileRetentionDuration - default 30 days

* Don't need to set them to be the same.  You may want to keep the log files around after the tombstoned files are purged.
* Time travel in order of months/years infeasible
* Initially desinged to correct mistakes

In [86]:
# files_in_dir(deltaPath,"parquet")
# Count files in deltaPath

numFiles = ! ls $deltaPath/*parquet | wc -l
print(f"There are {numFiles} files.")

There are ['1036'] files.


In [87]:
# Attempt to vacuum table with default settings

spark.sql("vacuum SalesDeltaFormat retain 0 hours dry run").show(100, truncate = False)

IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
       

In [88]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [89]:
%%time
# Vacuum Delta table to remove all history

spark.sql("VACUUM SalesDeltaFormat RETAIN 0 HOURS").show(truncate = False)
# ! Can use deltaTable.vacuum(0) against directory

+----------------------------------------------+
|path                                          |
+----------------------------------------------+
|file:/home/spark/data/delta/online_retail_data|
+----------------------------------------------+

CPU times: user 2.54 ms, sys: 5.19 ms, total: 7.74 ms
Wall time: 28.4 s


In [90]:
files_in_dir(deltaPath,"parquet")

195KB    2020-08-26 10:26:27  part-00003-fe509e92-fde4-4a82-b9f6-64a82125c721-c000.snappy.parquet
314KB    2020-08-26 10:26:27  part-00000-09673ae1-2d37-4bb5-af6d-195c2eb32120-c000.snappy.parquet
282KB    2020-08-26 10:26:27  part-00002-9a16b75f-9563-412c-8f74-6b3ffef17e9e-c000.snappy.parquet
312KB    2020-08-26 10:26:27  part-00001-9b64e2c8-dcdd-468a-856b-60d9349cc370-c000.snappy.parquet

Number of file/s: 4 | Total size: 1.6M


In [91]:
files_in_dir(deltaLogPath,"*")

2KB      2020-08-26 10:13:04  00000000000000000000.json
938B     2020-08-26 10:14:14  00000000000000000001.json
410B     2020-08-26 10:14:59  00000000000000000002.json
902B     2020-08-26 10:15:18  00000000000000000003.json
775B     2020-08-26 10:16:31  00000000000000000004.json
905B     2020-08-26 10:17:59  00000000000000000005.json
904B     2020-08-26 10:18:00  00000000000000000006.json
905B     2020-08-26 10:18:02  00000000000000000007.json
905B     2020-08-26 10:18:04  00000000000000000008.json
905B     2020-08-26 10:18:05  00000000000000000009.json
905B     2020-08-26 10:18:07  00000000000000000010.json
17KB     2020-08-26 10:18:08  00000000000000000010.checkpoint.parquet
2KB      2020-08-26 10:19:36  00000000000000000011.json
169KB    2020-08-26 10:25:15  00000000000000000012.json
141KB    2020-08-26 10:25:35  00000000000000000013.json
2KB      2020-08-26 10:26:28  00000000000000000014.json

Number of file/s: 16 | Total size: 400K


In [92]:
spark.read.format("json").load(deltaLogPath + "/00000000000000000014.json").collect()

[Row(add=None, commitInfo=Row(isBlindAppend=False, operation='WRITE', operationMetrics=Row(numFiles='4', numOutputBytes='1129731', numOutputRows='99226'), operationParameters=Row(mode='Overwrite', partitionBy='[]'), readVersion=13, timestamp=1598451987996, userMetadata='08-25-2020 Data Merge;'), remove=None),
 Row(add=Row(dataChange=True, modificationTime=1598451987000, path='part-00000-09673ae1-2d37-4bb5-af6d-195c2eb32120-c000.snappy.parquet', size=321712), commitInfo=None, remove=None),
 Row(add=Row(dataChange=True, modificationTime=1598451987000, path='part-00001-9b64e2c8-dcdd-468a-856b-60d9349cc370-c000.snappy.parquet', size=319754), commitInfo=None, remove=None),
 Row(add=Row(dataChange=True, modificationTime=1598451987000, path='part-00002-9a16b75f-9563-412c-8f74-6b3ffef17e9e-c000.snappy.parquet', size=288465), commitInfo=None, remove=None),
 Row(add=Row(dataChange=True, modificationTime=1598451987000, path='part-00003-fe509e92-fde4-4a82-b9f6-64a82125c721-c000.snappy.parquet', si

In [93]:
spark.read.format("delta").option("versionAsOf", currentVersion).load(deltaPath).count()

99226

In [94]:
# Configure Delta table to keep around 7 days of deleted data and 7 days of older log files
spark.sql("alter table SalesDeltaFormat set tblproperties ('delta.logRetentionDuration' = 'interval 7 days', 'delta.deletedFileRetentionDuration' = 'interval 7 days')")

DataFrame[]

In [95]:
# Verify our changes
spark.sql("describe extended SalesDeltaFormat").show(truncate = False)

+----------------------------+-----------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                      |comment|
+----------------------------+-----------------------------------------------------------------------------------------------+-------+
|InvoiceNo                   |int                                                                                            |       |
|StockCode                   |string                                                                                         |       |
|Description                 |string                                                                                         |       |
|Quantity                    |int                                                                                            |       |
|InvoiceDate                 |string                   

In [ ]:
# spark.stop()